# Colocalization App Tutorial

This tutorial will give you an idea of the backing algorithm to our Colocalization app by using a toy dataset.

This project allows researchers to colocalize eQTL hits from eQTLGen and GWAS hits from any study in openGWAS. It operates under the assumption that some GWAS and eQTL hits in close proximity may be the same signal, linking phenotypes to gene expression and enabling the development of transcriptional risk scores (TRS) and polygenic-predicted transcriptional risk scores (PP-TRS) for these phenotypes. As input, the project takes any study ID from <a href="https://www.opengwas.org/">openGWAS</a> as well as eQTL summary statistics from <a href="https://www.eqtlgen.org/cis-eqtls.html">eQTLGen</a>. The project has been implemented as an <a href="https://genapp2022.biosci.gatech.edu/team1/">app</a> using whole blood eQTL summary statistics, allowing users to download a list of GWAS and eQTL hits found to be the same signal. It was originally developed as a class project for BIOL8803 at Georgia Tech in Fall 2022.

Use this link to access the public Github which contains the necessary files to run the procedure locally and a detailed README: https://github.com/knishiura3/BIOL8803_Transcriptional_Risk_Score

## Setup
Ensure that you've set R as your kernel in Jupyter. See this tutorial for how to obtain an R kernel:
https://towardsdatascience.com/how-to-run-r-scripts-in-jupyter-15527148d2a

Run the following code blocks when the after completing the steps outlined in the README file on the command line. This ensures your conda environment is suitable for running the app. Verify that you have both the LD and MAF files locally as well.

This will also load the necessary packages from that environment.

In [ ]:
%%R
# packages for parquet querying
library(arrow)
library(duckdb)
library(fs)
library(tidyverse)
library(DBI)
library(glue)

# packages for gassocplot
library(grid)
library(gridExtra) 
library(gtable)


# packages for coloc
suppressPackageStartupMessages(suppressWarnings({
    library(gwasglue)
    library(gassocplot)
    library(dplyr)
    library(coloc)
    library(ggplot2)
    library(httpgd)

}))

## Define helper functions
Within the app, these are sourced from three separate R files.

In [ ]:
#from eQTL_query_parquet.r
initialize_db <- function() {
    # open parquet files
    ds_eQTL <- arrow::open_dataset(dir_eqtl, partitioning = "SNPChr")
    ds_eqtlMAF <- arrow::open_dataset(dir_eqtlmaf, partitioning = "hg19_chr")

    # clean up if previous connection if still open (only happens if script is interrupted)
    if (exists("con")) {
        duckdb::duckdb_unregister(con, "eqtlTable")
        duckdb::duckdb_unregister(con, "mafTable")
        duckdb::dbDisconnect(con)
    }

    # connect duckdb to open parquet files
    con <- duckdb::dbConnect(duckdb::duckdb())

    # register the datasets as DuckDB table, and give it a name
    duckdb::duckdb_register_arrow(con, "eqtlTable", ds_eQTL)
    duckdb::duckdb_register_arrow(con, "mafTable", ds_eqtlMAF)
    return(con)
}

gather_and_format_gwas_eqtl_in_region <- function(con, chromosome, gwas_hit) {
    # str(gwas_hit)
    # get the position of the current GWAS hit
    gwas_pos <- gwas_hit$position
    lower <- gwas_pos - window_size
    # set floor of 0 for lower bound of window
    if (lower < 0) {
        lower <- 0
    }
    upper <- gwas_pos + window_size
    chrpos <- paste0(chromosome, ":", lower, "-", upper)

    # window function query for top eQTLs, defined as:
    #   for each locus, determined by values of chromosome & position,
    #   sort descending by absolute value Zcore and take only top row)
    subquery_eqtl <- glue::glue(
        "
        SELECT * FROM(
            SELECT *, ROW_NUMBER() OVER (PARTITION BY SNPPos ORDER BY abs(Zscore) DESC) AS 'row'
            FROM eqtlTable
            WHERE SNPChr = {chromosome} AND SNPPos BETWEEN {lower} AND {upper}
            )
        WHERE row = 1
        "
    )
    subquery_maf <- glue::glue(
        "
        SELECT * FROM mafTable
        WHERE hg19_chr = {chromosome}
        "
    )
    # execute the SQL queries
    result_eqtl <- dbGetQuery(con, subquery_eqtl)
    result_maf <- dbGetQuery(con, subquery_maf)
    print('SQL queries executed')
    # left join the eQTL and MAF tables to get full set of relevant eqtl data
    result_raw <- left_join(result_eqtl, result_maf,
        by = c("SNPChr" = "hg19_chr", "SNPPos" = "hg19_pos"),
        suffix = c(".eqtl", ".maf")
    ) 
    print('join complete')
    # filter and rename columns
    result <- result_raw %>%
        # keep columns Pvalue, SNP, SNPPos, Zscore, NrSamples, AlleleB_all
        dplyr::select(Pvalue, NrSamples, AlleleB_all, SNP.eqtl, Zscore, SNPPos, AssessedAllele,OtherAllele) %>%
        # add back column SNPChr
        tibble::add_column(chr = as.character(chromosome), .before = "SNPPos") %>%
        # rename AlleleB_all to MAF
        dplyr::rename(eaf = AlleleB_all, rsid = SNP.eqtl, p = Pvalue, n = NrSamples, z = Zscore, position = SNPPos, ea = AssessedAllele, nea = OtherAllele)
    
    # str(result)
    # harmonize gwas and eqtl datasets and structure for downstream analysis  
    out <- ieugwasr_to_coloc_custom(
        id1= gwas_dataset, 
        result, # eqtl data
        type2 = "quant", 
        chrompos = as.character(chrpos)
    )
    
    # str(out)
    # run coloc
    res <- coloc::coloc.abf(out[[1]], out[[2]])
    # str(res)
    # extract H4 from coloc output
    PP_H4 <- round(as.numeric(res$summary[["PP.H4.abf"]]), digits = 2)
    
    return(list(out, PP_H4, result_raw))
}

extract_top_markers_from_gassocplot_output <- function(temp) {
    # str(temp)
    # code modified from gassocplot for identifying top markers
    # https://github.com/jrs95/gassocplot/blob/master/R/figures.R#L387-L390
    mlog10p_gwas <- -(log(2) + pnorm(-abs(temp$z[[1]]), log.p=TRUE))/log(10)
    mlog10p_eqtl <- -(log(2) + pnorm(-abs(temp$z[[2]]), log.p=TRUE))/log(10)
    # add as column of temp object because of shared indices
    temp$stats_gwas <- mlog10p_gwas
    temp$stats_eqtl <- mlog10p_eqtl
    # get the values and indices of the the maximum value of temp$stats (or elements of values/indices tied for max)
    max_pval_gwas <- max(temp$stats_gwas)
    max_pval_eqtl <- max(temp$stats_eqtl)
    # str(max_pval_eqtl)
    max_pval_indices_gwas <- which(temp$stats_gwas == max_pval_gwas)
    max_pval_indices_eqtl <- which(temp$stats_eqtl == max_pval_eqtl)
    # str(max_pval_indices_eqtl)
    # get the rsid (temp$markers$marker) at the index position of top marker
    top_marker_gwas <- temp$markers$marker[max_pval_indices_gwas]
    top_marker_eqtl <- temp$markers$marker[max_pval_indices_eqtl]

    return(list(top_marker_gwas, top_marker_eqtl))
}

# take top markers as input and return a raw data row for each colocalized gwas and eqtl pair
get_top_marker_raw_data <- function(top_marker_gwas, top_marker_eqtl, result_raw, PP_H4) {
    # create a table of the row in result_raw that corresponds to the top marker and append H4 as a column
    # and only keep desired columns
    top_eqtl_table <- result_raw[result_raw$SNP.eqtl %in% top_marker_eqtl, c("GeneChr","Pvalue", "SNP.eqtl", "SNPPos", "AssessedAllele", "OtherAllele", "Zscore", "Gene", "GenePos", "NrCohorts", "NrSamples", "FDR", "BonferroniP", "AlleleA", "AlleleB", "allA_total", "allAB_total", "allB_total", "AlleleB_all")]
    # rename GeneChr to Chr
    colnames(top_eqtl_table)[colnames(top_eqtl_table) == "GeneChr"] <- "Chr"
    # loop over and add .eqtl suffix to columns Pvalue, SNPPos, AssessedAllele, OtherAllele, Zscore, AlleleA, AlleleB, allA_total, allAB_total, allB_total, AlleleB_all
    for (i in 1:ncol(top_eqtl_table)) {
        # skip column name that already has .eqtl suffix or if Chr
        if (grepl(".eqtl", colnames(top_eqtl_table)[i]) | colnames(top_eqtl_table)[i] == "Chr") {
            next
        }
        colnames(top_eqtl_table)[i] <- paste0(colnames(top_eqtl_table)[i], ".eqtl")
    }

    # queries GWAS table w/ rsid for top marker
    top_gwas <- ieugwasr::associations(top_marker_gwas,gwas_dataset)
    # only keep columns position, beta, se, p, n, id, rsid, ea, nea, eaf, trait
    top_gwas <- top_gwas[, c("position", "beta", "se", "p", "n", "id", "rsid", "ea", "nea", "eaf", "trait")]
    colnames(top_gwas) <- paste0(colnames(top_gwas), ".gwas")
    # combine top_gwas and top_eqtl_table side-by-side in new table
    top_table <- cbind(top_gwas, top_eqtl_table)
    # move Chr column to first position
    top_table <- top_table[, c("Chr", colnames(top_table)[!colnames(top_table) %in% "Chr"])]
    top_table$H4 <- PP_H4
    return(top_table)
}

plot_associated_signals_and_save <- function(tophit_idx, chromosome, PP_H4, temp, plot_dir) {
    # this is needed to suppress automatic pdf output from gassocplot
    pdf(NULL)
    # construct plot
    theplot <- stack_assoc_plot_custom(temp$markers, temp$z, temp$corr, traits = temp$traits)
    # output path for saving a figure for each colocalized pair of GWAS/eQTL 
    outfile <- glue::glue(plot_dir, "/chr{chromosome}_gwastophit{sprintf('%03d', tophit_idx)}_H4_{PP_H4}.png")
    # if it doesn't exist, create directory
    if (!dir.exists(plot_dir)) {
        dir.create(plot_dir, recursive = TRUE)
    }
    # save plot w/ base R functions
    
    png(filename = outfile)
    plot(theplot)
    dev.off()
}

# function to clean up
clean_up <- function(con) {
    # clean up
    print("cleaning up")
    duckdb_unregister(con, "eqtlTable")
    duckdb_unregister(con, "mafTable")
    dbDisconnect(con, shutdown = TRUE)
    print("done")
}


#from ieugwasr_to_coloc.r
ieugwasr_to_coloc_custom <- function(id1, eqtl_df, chrompos, type1=NULL, type2=NULL)
{
    id2 <- 'eQTL-Gen'
	tab1 <- ieugwasr::associations(id=id1, variants=chrompos) %>% subset(., !duplicated(rsid))
	# tab2 <- ieugwasr::associations(id=id2, variants=chrompos) %>% subset(., !duplicated(rsid))
	tab2 <- eqtl_df %>% subset(., !duplicated(rsid)) # tab2 is the eQTL dataset
	commonsnps <- tab1$rsid[tab1$rsid %in% tab2$rsid]
	tab1 <- tab1[tab1$rsid %in% commonsnps, ] %>% dplyr::arrange(rsid)
	tab2 <- tab2[tab2$rsid %in% commonsnps, ] %>% dplyr::arrange(rsid)
	stopifnot(all(tab1$rsid == tab2$rsid))

	index <- as.character(tab1$ea) == as.character(tab2$ea) &
			as.character(tab1$nea) == as.character(tab2$nea) &
			as.character(tab1$rsid) == as.character(tab2$rsid) &
			tab1$position == tab2$position
	stopifnot(sum(index) > 0)
	tab1$eaf <- as.numeric(tab1$eaf)
	tab2$eaf <- as.numeric(tab2$eaf)
	tab1$eaf[which(tab1$eaf > 0.5)] <- 1 - tab1$eaf[which(tab1$eaf > 0.5)]
	tab2$eaf[which(tab2$eaf > 0.5)] <- 1 - tab2$eaf[which(tab2$eaf > 0.5)]
	s <- sum(is.na(tab1$eaf))
	if(s > 0)
	{
		warning(s, " out of ", nrow(tab1), " variants have missing allele frequencies in ", id1, ". Setting to 0.5")
		tab1$eaf[is.na(tab1$eaf)] <- 0.5
	}
	s <- sum(is.na(tab2$eaf))
	if(s > 0)
	{
		warning(s, " out of ", nrow(tab2), " variants have missing allele frequencies in ", id2, ". Setting to 0.5")
		tab2$eaf[is.na(tab2$eaf)] <- 0.5
	}

	info1 <- ieugwasr::gwasinfo(id1)
	type1 <- get_type(info1, type1)
	# info2 <- ieugwasr::gwasinfo(id2)
	type2 <- "quant"


	tab1$n[is.na(tab1$n)] <- info1$sample_size
    # swap any NA values in eqtl table w/ median sample size
    tab2$n[is.na(tab2$n)] <- median(tab2$n)

	tab1 <- tab1[index,] %>% {list(pvalues = .$p, N = .$n, MAF = .$eaf, beta = .$beta, varbeta = .$se^2, type = type1, snp = .$rsid, z = .$beta / .$se, chr = .$chr, pos = .$position, id = id1)}
	tab2 <- tab2[index,] %>% {list(pvalues = .$p, N = .$n, MAF = .$eaf, 
        # beta = NULL, varbeta = NULL, 
        type = type2, snp = .$rsid, z = .$z, chr = .$chr, pos = .$position, id = id2)}

	if(type1 == "cc")
	{
		tab1$s <- info1$ncase / info1$sample_size
	}

	if(type2 == "cc")
	{
		tab2$s <- info2$ncase / info2$sample_size
	}

	return(list(dataset1=tab1, dataset2=tab2))
}

get_type <- function(info, typex)
{
	if(!is.null(typex))
	{
		stopifnot(typex %in% c("cc", "quant"))
		return(typex)
	} else if(is.na(info$unit)) {
		if(! "ncase" %in% names(info))
		{
			info$ncase <- NA
		}
		if(is.na(info$ncase))
		{
			message("Type information not available for ", info$id, ". Assuming 'quant' but override using 'type' arguments.")
			return("quant")			
		} else {
			message("No units available but assuming cc due to number of cases being stated")
			return("cc")
		}
	} else {
		return(ifelse(info$unit %in% c("logOR", "log odds"), "cc", "quant"))
	}
}


#from gassocplot.r
g_legend<-function(gplot){
  tmp <- ggplot_gtable(ggplot_build(gplot))
  leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box")
  legend <- tmp$grobs[[leg]]
  return(legend)
}

add_g_legend <- function(g, legend){
  lheight <- sum(legend$height)*1.5
  g <- grid.arrange(g, legend, ncol = 1, heights = unit.c(unit(1, "npc") - lheight, lheight))
  return(g)
}

stack_assoc_plot_custom <- function(markers, z, corr=NULL, corr.top=NULL, traits, ylab=NULL, type="log10p", x.min=NULL, x.max=NULL, top.marker=NULL, legend=TRUE){
  
  # Error messages
  if(!(type=="log10p" | type=="prob")) stop("the type of plot has to be either log10p or prob")
  if(length(traits)!=ncol(z)) stop("the number of traits is not the same as the number of columns for the Z-scores")
  if(nrow(markers)!=nrow(z)) stop("the number of markers is not the same as the number of rows for the Z-scores")
  if(!is.null(corr)){if(ncol(corr)!=nrow(markers) | nrow(corr)!=nrow(markers)) stop("corr has to have the same dimensions as the number of rows in the markers dataset")}
  if(!is.null(corr.top)){if(length(corr.top)!=nrow(markers)) stop("corr.top has to have the same length as the number of rows in the markers dataset")}
  # if(any(rownames(corr)!=markers$marker)) stop("corr has to have the same markers in the same order as the markers dataset")
  if(any(names(markers)!=c("marker", "chr", "pos"))) stop("dataset needs to include marker, chr and pos columns in that order")
  if(length(unique(markers$chr))>1) stop("there should only be markers from one chromosome in the markers dataset")   
  if(!(markers$chr[1] %in% 1:22)) stop("the plotting tool is only for autosomal chromosomes") 
  if(any(is.na(markers))) stop("there are missing markers in your marker dataset") 
  # if(any(is.na(z))) stop("there are missing values in the Z-score matrix")
  if(class(markers$pos)!="integer") stop("the pos variable has to be an integer")
  if(is.null(corr) & !is.null(corr.top) & is.null(top.marker)) stop("top.marker must be defined if corr.top is provided")
  if(is.null(corr) & !is.null(corr.top)){if(length(corr.top)!=nrow(markers)) stop("corr.top has to have the same length as the number of rows in the markers dataset")}
  if(!is.null(top.marker) & length(which(top.marker==markers$marker))==0) stop("top.marker is not contained in the markers dataset")
  if(!is.null(top.marker) & length(which(top.marker==markers$marker))>1) stop("top.marker maps to multiple markers in the markers dataset")
  
  # Coerce data
  markers$marker <- as.character(markers$marker)
  chr <- as.integer(markers$chr[1])
  r2_legend <- legend
  if(is.null(x.min)){x.min <- min(as.integer(markers$pos))}
  if(is.null(x.max)){x.max <- max(as.integer(markers$pos))}
  if((x.max - x.min)>10000000) stop("the plotting tool can plot a maximum of 10MB")
  
  # mlog10p
  if(type=="log10p"){
    mlog10p <- suppressWarnings(apply(z, 2, function(x){-(log(2) + pnorm(-abs(x), log.p=T))/log(10)}))
    # Kenji Nishiura:  don't impose a -log10(pval) ceiling 
    # mlog10p[mlog10p>1000 & !is.na(mlog10p)] <- 1000
  }
  if(type=="log10p"){ylab <- expression("-log"["10"]*paste("(",italic("p"),")"))}else{if(is.null(ylab)){ylab <- "Probability"}}
 
  # Genes
  gene.region <- gassocplot::genes[gassocplot::genes$chr==chr & !(gassocplot::genes$end<x.min) & !(gassocplot::genes$start>x.max),]
  gene.region$start[gene.region$start<x.min] <- x.min
  gene.region$end[gene.region$end>x.max] <- x.max
  gene.region <- gene.region[with(gene.region, order(start)), ]
  ngenes <- nrow(gene.region)

  # Max and min
  x.min <- x.min - 0.02*(x.max - x.min)
  x.max <- x.max + 0.02*(x.max - x.min)
  
  # Correlation matrix
  if(is.null(corr) & is.null(corr.top)){r2_legend <- FALSE; corr <- matrix(NA, nrow=nrow(markers), ncol=nrow(markers))}

  # Recombination plot
  recombination.plot <- plot_recombination_rate_stack(chr, x.min, x.max)

  # Gene plot
  if(ngenes==0){gene.plot <- plot_gene_zero(chr, x.min, x.max, stack=TRUE)}
  if(ngenes>0 & ngenes<=5){gene.plot <- plot_gene_two(gene.region, chr, x.min, x.max, stack=TRUE)}
  if(ngenes>5 & ngenes<=10){gene.plot <- plot_gene_five(gene.region, chr, x.min, x.max, stack=TRUE)}
  if(ngenes>10 & ngenes<=25){gene.plot <- plot_gene_ten(gene.region, chr, x.min, x.max, stack=TRUE)}
  if(ngenes>25){gene.plot <- plot_gene_fifteen(gene.region, chr, x.min, x.max, stack=TRUE)}

  # Top marker
  if(length(top.marker)!=0){if(is.na(top.marker)){top.marker <- NULL}}
    
  # Association plot
  for(i in length(traits):1){
    if(type=="log10p"){
      data <- data.frame(marker=markers$marker, chr=as.integer(markers$chr), pos=as.integer(markers$pos), stats=mlog10p[,i], stringsAsFactors=F)
    }else{
      data <- data.frame(marker=markers$marker, chr=as.integer(markers$chr), pos=as.integer(markers$pos), stats=z[,i], stringsAsFactors=F)    
    }
    marker.plot <- plot_assoc_stack(data, corr, corr.top, x.min, x.max, top.marker, ylab, type)
    legend <- g_legend(marker.plot)
    if(i==length(traits)){g <- plot_regional_gene_assoc(recombination.plot, marker.plot, gene.plot, traits[i], ngenes)}
    if(i<length(traits)){
      g1 <- plot_regional_assoc(recombination.plot, marker.plot, traits[i])
      g <- gtable:::rbind_gtable(g1, g, "last")
      panels <- g$layout$t[grep("panel", g$layout$name)]
      g$heights[panels[1]] <- unit(3,"null") 
    } 
  }

  # Combined plot
  if(r2_legend==T){
    combined.plot <- add_g_legend(g, legend)
  }else{
    combined.plot <- g
  }

  return(combined.plot)
}

## Collect Inputs
Within the app, this is done reactively in R Shiny whenever an interactive field is changed and the submit button is pressed. In this tutorial, we'll use ieu-b-30 as the example GWAS ID and focus on chromosome 21.

In [ ]:
gwas_dataset <<- "ieu-b-30"
chromosomes <<- 21
dir_ld <<- "ld"
dir_eqtl <<- "data/eqtls"
dir_eqtlmaf <<- "data/eqtl_MAF"
eqtl_outdir <<- "top_eqtls"
plot_dir <<- "plots"

# if output subdirectories don't exist yet, create them
if (!dir.exists(eqtl_outdir)) {
    dir.create(eqtl_outdir)
}
if (!dir.exists(plot_dir)) {
    dir.create(plot_dir)
}

# if it doesn't exist, create a directory named user_outdir in plots and top_eqtls
# if it exists already, delete files from previous run
# if user_outdir is defined and the plot_dir/user_outdir exists, delete the files in that directory
if (exists("user_outdir") && dir.exists(glue::glue(plot_dir,'/', user_outdir))) {
    unlink(glue::glue(plot_dir,'/', user_outdir, '/*'))
    unlink(glue::glue(eqtl_outdir,'/', user_outdir, '/*'))
}

# generate unique id using timestamp
user_outdir <<- as.character(Sys.time())
user_outdir <<- gsub(" ", "_", user_outdir)
user_outdir <<- gsub(":", "_", user_outdir)
user_outdir <<- gsub("-", "_", user_outdir)

# if user subdirectories don't exist yet, create them
if (!dir.exists(glue::glue(plot_dir,'/', user_outdir))) {
    dir.create(glue::glue(plot_dir,'/', user_outdir))
}
if (!dir.exists(glue::glue(eqtl_outdir,'/', user_outdir))) {
    dir.create(glue::glue(eqtl_outdir,'/', user_outdir))
}


window_size <<- as.integer(100000)
con <- initialize_db()
# query API for top GWAS hits for all chromosomes
top_all <- ieugwasr::tophits(gwas_dataset)

# initialize counter at 0 each time submit button is pressed
colocalized_counter <- 0

## Run colocalization analysis on each chromosome


In [ ]:
for (chromosome in chromosomes) {    
    # filter the top GWAS hits for the current chromosome
    top <- top_all %>%
        filter(chr == chromosome) %>%
        arrange(p)
    # initialize progressbar for each chromosome separately
#     withProgress(
#         message='Please wait',
#         detail=glue::glue('Processing chr {chromosome}:'),
#         value=0, {
    for (tophit_idx in seq_len(nrow(top))) {
        # calls customized ieugwasr_to_coloc function to take in eQTL data as direct input
        out_PPH4_rawResult <- gather_and_format_gwas_eqtl_in_region(con, chromosome, top[tophit_idx,])
        # unpack returned objects
        out <- out_PPH4_rawResult[[1]]
        PP_H4 <- out_PPH4_rawResult[[2]]
        # continue to the next window if posterior probability of H4 is less than 0.5
        if (PP_H4 < 0.50) {
            print(glue::glue("PP_H4 < 0.50, skipping and continuing to next GWAS top hit"))
            setProgress(tophit_idx / nrow(top), detail = glue::glue('Processing chr {chromosome}: region {tophit_idx} out of {nrow(top)}'))
            next
        }
        # API rejects requests if >500 rsids, so need to run plink locally
        if (length(out[[2]]$pos) >= 500) {
            print("too many rsids, running plink locally")
            # input to coloc_to_gassocplot is list of rsids (should be identical in gwas/eqtl data at this stage): out[[1]]$snp
            # choices for ancestry are AMR, AFR, EAS, EUR, SAS
            # note: a bit slow first time because the plink_bin function will download/install plink if it's not already installed.
            temp <- coloc_to_gassocplot(out, bfile = paste0(dir_ld, "/EUR"), plink_bin = '/projects/team1/bin/mambaforge/envs/plink/bin/plink')
            # local debugging (comment out line above, uncomment line below)
            # temp <- coloc_to_gassocplot(out, bfile = paste0(dir_ld, "/EUR"), plink_bin = genetics.binaRies::get_plink_binary())
            # increment counter for number of colocalized eQTLs
            colocalized_counter <- colocalized_counter + 1
        } else {
            print("running coloc_to_gassocplot via API")
            # query the API if <500 rsids
            temp <- coloc_to_gassocplot(out)
            # increment counter for number of colocalized eQTLs
            colocalized_counter <- colocalized_counter + 1
        }
        # get the rsids matching the gassocplot plot labels
        top_marker_gwas <- extract_top_markers_from_gassocplot_output(temp)[1]
        top_marker_eqtl <- extract_top_markers_from_gassocplot_output(temp)[2]

        result_raw <- out_PPH4_rawResult[[3]]

        top_table <<- get_top_marker_raw_data(top_marker_gwas, top_marker_eqtl, result_raw, PP_H4)
        top_table_filename <<- "eQTLs_colocalized_w_GWAS.txt"
        top_table_path <<- glue::glue(eqtl_outdir, '/', user_outdir, '/', top_table_filename)
        # print(top_table_path)
        # write the header only once
        if (!file.exists(top_table_path)) {
            write.table(top_table, top_table_path, sep = "\t", row.names = FALSE, quote = FALSE,col.names = TRUE, append = FALSE)
        } else {
            write.table(top_table, top_table_path, sep = "\t", row.names = FALSE, quote = FALSE, col.names = FALSE, append = TRUE)
        }    
        # call customized gassocplot function w/ -log(pval) ceiling removed
        plot_associated_signals_and_save(tophit_idx, chromosome, PP_H4, temp, glue::glue(plot_dir,'/', user_outdir))


        # increment progress bar
        print(glue::glue('working on tophit_idx {tophit_idx} out of {nrow(top)}'))
        setProgress(tophit_idx / nrow(top), detail = glue::glue('Processing chr {chromosome}: region {tophit_idx} out of {nrow(top)}'))

        } # close loop over each gwas top hit
    # redraw the main panel explicitly
    imgs <<- list.files(glue::glue(plot_dir,'/', user_outdir), pattern=".png", full.names = TRUE)
    # str(imgs)
    output[["slickr"]] <<- renderSlickR({
        slickR(imgs) + settings(slidesToShow = 1, slidesToScroll = 1, lazyLoad = 'anticipated', dots = TRUE, arrows = TRUE, infinite = FALSE)
    })
#     }) # close withProgress
} # close loop over chromosomes

clean_up(con) 

## Output resulting plots
The image files will be stored in the plots.zip directory. In the app, this file is provided through a download button. For the purposes of this tutorial, one of the example plots in the directory will be printed.

In [ ]:
# if colocalized_counter is 0, print message
if (colocalized_counter == 0) {
    print("No colocalized eQTLs found. Try again with different chromosomes or GWAS datasets.")
    
} else {
    print(glue::glue("Number of colocalized eQTLs: {colocalized_counter}"))
}

plot_names <- basename(imgs)
# zip plots and store in user directory
zip::zip(zipfile = 'plots.zip', files = plot_names, root = glue::glue(plot_dir,'/', user_outdir), recurse = FALSE)
zip_path <<- glue::glue(plot_dir,'/', user_outdir, '/plots.zip')
        
        
#Show in notebook one of the gassocplots

# From previous coloc_pipeline.ipynb (to be deleted if unneeded once current algorithm is outlined.)

In [ ]:
%%R
# dir_eqtl <- "eqtls"
dir_eqtlmaf <- "eqtl_MAF"

# open parquet files
# ds_eQTL <- arrow::open_dataset(dir_eqtl, partitioning = "SNPChr")
ds_eqtlMAF <- arrow::open_dataset(dir_eqtlmaf, partitioning = "hg19_chr")

con <- dbConnect(duckdb::duckdb())

# register the dataset as a DuckDB table, and give it a name
# duckdb::duckdb_register_arrow(con, "eqtlTable", ds_eQTL)
duckdb::duckdb_register_arrow(con, "mafTable", ds_eqtlMAF)

# read in the file
# query the table for each chromosome separately
for (chr in 1:22) {
    
    # Left join result and 2018-07-18_SNP_AF_for_AlleleB_combined_allele_counts_and_MAF_pos_added.txt  on SNPChr and SNPPos
    query <- glue::glue(
        "
        SELECT SNP, hg19_pos
        FROM 'eqtl_MAF/hg19_chr={chr}/*.parquet'
        "
    )
    tic()
    result <- dbGetQuery(con, query)
    toc()
    # and print the first 10 rows
    print(head(result, 10))
}
# clean up
duckdb_unregister(con, "eqtlTable")
duckdb_unregister(con, "mafTable")
dbDisconnect(con)

In [ ]:
# wbc count related to autoimmune disease
gwas_dataset <- "ieu-b-30"
dummy_dataset <- "ieu-a-7"
gwasinfo(id = as.character(gwas_dataset))
# gwasinfo(id=as.character(dummy_dataset))


In [ ]:
# queries API from https://gwas.mrcieu.ac.uk/ to:
    # 1) grab VCF file and 
    # 2) identify top GWAS hits
    # 3) sort by p-value order (lowest to highest)
top <- ieugwasr::tophits("ieu-b-30") %>% arrange(p)
print(paste0(dim(top)[1], " GWAS hits identified"))
head(top)


In [ ]:
window_size <- 50000
if (!dir.exists("gwas_hits")) {
    dir.create("gwas_hits")
}

# parallel loop over the top GWAS hits in p-value order and iterate over rows
foreach(i = 1:nrow(top)) %dopar% {
    # str(top[i,])
    # add leading 0s to pval_rank_gwashit so that it always has 3 digits
    pval_rank_gwashit <- sprintf("%03d", i)
    chr_gwashit <- top[i, ]$chr
    pos_gwashit <- top[i, ]$position

    # print(paste0("Processing GWAS hit ", pval_rank_gwashit, " of ", nrow(top)))

    lower <- pos_gwashit - window_size
    upper <- pos_gwashit + window_size
    # print(paste(pval_rank_gwashit," ",chr_gwashit," ",lower," ",upper))
    chrpos <- paste0(chr_gwashit, ":", lower, "-", upper)
    # print(chrpos)

    # extract SNPS in the region of top hit
    out <- ieugwasr_to_coloc(
        id1 = as.character(gwas_dataset),
        id2 = as.character(dummy_dataset),
        chrompos = as.character(chrpos),
        type1 = "quant",
        # type2 = "cc"
    )

    # get rid of dummy dataset
    out <- out[1]$dataset1
    # str(out)

    # export to file (tab-delimited) in directory named gwas_hits in the following format:
    # <pval rank>_<chr>_<lower bound>_<upper bound>.tsv
    write.table(out,
        file = paste0(
            "gwas_hits/", pval_rank_gwashit, "_", chr_gwashit, "_", lower, "_", upper, "_gwas.tsv"
        ),
        sep = "\t", quote = FALSE, row.names = FALSE, col.names = TRUE
    )
}


In [ ]:
# Build eQTL DB
# python3 eQTL_build_db.py

# Query eQTL DB; Change integer in "-P0" to specify the number of threads
# find gwas_hits/ -type f -name "*" | \
# xargs -P0 -I{} python3 eQTL_query_db.py --input_file {} --db_name eQTLs.db --output_dir eqtl_hits
    # can check gnomad if maf assumptions are reasonable

In [ ]:
# gather up all the gwas and eqtl files
gwas_file_list <- list.files(path = "gwas_hits", pattern = ".tsv", full.names = TRUE)
eqtl_file_list <- list.files(path = "output_full", pattern = ".tsv", full.names = TRUE)

# str(gwas_file_list)
# str(eqtl_file_list)

# make a directory named output_coloc if it doesn't exist
dir.exists("output_coloc") || dir.create("output_coloc")

for (i in seq_along(gwas_file_list)) {

    # loop over elements in gwas_file_list and eqtl_file_list at the same time
    gwas_file <- gwas_file_list[i]
    eqtl_file <- eqtl_file_list[i]

    gwas_data <- as.list(read.table(file = gwas_file, header = TRUE, sep = "\t"))

    # working around formatting issues
    eqtl_data <- read.table(file = eqtl_file, header = FALSE, sep = "\t", fill = TRUE)
    # drop the last column in eqtl_data - not sure why there is an empty column read in from the file???
    eqtl_data <- eqtl_data[, -ncol(eqtl_data)]
    # rename the columns by first element
    colnames(eqtl_data) <- eqtl_data[1, ]
    # drop the first row
    eqtl_data <- eqtl_data[-1, ]
    # set column data types manually
    eqtl_data$pvalues <- as.numeric(eqtl_data$pvalues)
    eqtl_data$N <- as.integer(eqtl_data$N)
    eqtl_data$MAF <- as.numeric(eqtl_data$MAF)
    eqtl_data$beta <- as.numeric(eqtl_data$beta)
    eqtl_data$varbeta <- as.numeric(eqtl_data$varbeta)
    eqtl_data$type <- as.character(eqtl_data$type)
    eqtl_data$snp <- as.character(eqtl_data$snp)
    eqtl_data$z <- as.numeric(eqtl_data$z)
    eqtl_data$chr <- as.integer(eqtl_data$chr)
    eqtl_data$pos <- as.integer(eqtl_data$pos)
    eqtl_data$id <- as.character(eqtl_data$id)

    eqtl_data <- as.list(eqtl_data)

    # run coloc
    res <- coloc::coloc.abf(gwas_data, eqtl_data)

    #  Get locus from filenames
    SNP_locus_name <- basename(gwas_file)
    SNP_locus_name <- sub("_gwas\\..*$", "", SNP_locus_name)
    # split on _ assign first element to pval_rank, second to chr, third to lower bound, fourth to upper bound
    SNP_locus_name <- strsplit(SNP_locus_name, "_")
    # flatten to vector
    SNP_locus_name <- unlist(SNP_locus_name)
    pval_rank <- SNP_locus_name[1]
    chr <- SNP_locus_name[2]
    lower <- SNP_locus_name[3]
    upper <- SNP_locus_name[4]


    # write posterior probability output to separate log files
    # if H4 >= 80%, send to one file
    if (as.numeric(res$summary["PP.H4.abf"]) >= 0.8) {
        write.table(paste(pval_rank, chr, lower, upper, as.numeric(res$summary["PP.H1.abf"]), as.numeric(res$summary["PP.H2.abf"]), as.numeric(res$summary["PP.H3.abf"]), as.numeric(res$summary["PP.H4.abf"])),
            file = paste0(
                "output_coloc/H4gt80.tsv"
            ),
            sep = "\t", quote = FALSE, row.names = FALSE, col.names = FALSE, append = TRUE
        )
        # if H4 < 80%, send to another file
    } else if (as.numeric(res$summary["PP.H4.abf"]) < 0.8) {
        write.table(paste(pval_rank, chr, lower, upper, as.numeric(res$summary["PP.H1.abf"]), as.numeric(res$summary["PP.H2.abf"]), as.numeric(res$summary["PP.H3.abf"]), as.numeric(res$summary["PP.H4.abf"])),
            file = paste0(
                "output_coloc/H4lt80.tsv"
            ),
            sep = "\t", quote = FALSE, row.names = FALSE, col.names = FALSE, append = TRUE
        )
    }
    # write.table(paste(pval_rank,chr,lower,upper,as.numeric(res$summary["PP.H1.abf"]),as.numeric(res$summary["PP.H2.abf"]),as.numeric(res$summary["PP.H3.abf"]),as.numeric(res$summary["PP.H4.abf"])), file = paste0("output_coloc/", "posteriors.tsv"), sep = "\t", quote=FALSE, row.names = FALSE, col.names = FALSE, append = TRUE)
}


In [ ]:
# get the highest priors from the H4 < 80% list to a dataframe named posteriors
posteriors <- read.table(file = "output_coloc/H4lt80.tsv", header = FALSE, sep = " ")

# manually name columns
colnames(posteriors) <- c("pval_rank", "chr", "lower", "upper", "PP.H1.abf", "PP.H2.abf", "PP.H3.abf", "PP.H4.abf")

# sort posteriors by the last column in descending order
posteriors <- posteriors[order(-posteriors$PP.H4.abf), ]

# only consider top 10 H4 posterior probabilities and identify gwas and eqtl rows they came from
top10H4 <- posteriors[1:10, ]
head(top10H4, n = 10L)


# manually set the gwas and eqtl file names by concatenating the values of the first 4 columns in top5H4
gwas_files <- paste0("gwas_hits/", top10H4$pval_rank, "_", top10H4$chr, "_", top10H4$lower, "_", top10H4$upper, "_gwas.tsv")
eqtl_files <- paste0("eqtl_hits/", top10H4$pval_rank, "_", top10H4$chr, "_", top10H4$lower, "_", top10H4$upper, "_eQTLs.tsv")

# loop over records and construct out object for plotting
for (i in seq_along(gwas_files)) {

    # loop over elements in gwas_file_list and eqtl_file_list at the same time
    gwas_file <- gwas_files[i]
    eqtl_file <- eqtl_files[i]

    gwas_data <- read.table(file = gwas_file, header = TRUE, sep = "\t")

    # working around formatting issues
    eqtl_data <- read.table(file = eqtl_file, header = FALSE, sep = "\t", fill = TRUE)
    # drop the last column in eqtl_data - not sure why there is an empty column read in from the file???
    eqtl_data <- eqtl_data[, -ncol(eqtl_data)]
    # rename the columns by first element
    colnames(eqtl_data) <- eqtl_data[1, ]
    # drop the first row
    eqtl_data <- eqtl_data[-1, ]
    # set column data types manually
    eqtl_data$pvalues <- as.numeric(eqtl_data$pvalues)
    eqtl_data$N <- as.integer(eqtl_data$N)
    eqtl_data$MAF <- as.numeric(eqtl_data$MAF)
    eqtl_data$beta <- as.numeric(eqtl_data$beta)
    eqtl_data$varbeta <- as.numeric(eqtl_data$varbeta)
    eqtl_data$type <- as.character(eqtl_data$type)
    eqtl_data$snp <- as.character(eqtl_data$snp)
    eqtl_data$z <- as.numeric(eqtl_data$z)
    eqtl_data$chr <- as.integer(eqtl_data$chr)
    eqtl_data$pos <- as.integer(eqtl_data$pos)
    eqtl_data$id <- as.character(eqtl_data$id)

    # subset gwas_data and eqtl_data for shared values of chr and pos, drop rows that don't exist in BOTH
    gwas_data <- gwas_data[gwas_data$chr %in% eqtl_data$chr & gwas_data$pos %in% eqtl_data$pos, ]
    eqtl_data <- eqtl_data[eqtl_data$chr %in% gwas_data$chr & eqtl_data$pos %in% gwas_data$pos, ]

    # create a list named out with two elements named dataset1 and dataset 2 and append gwas_data and eqtl_data
    out <- list(dataset1 = as.list(gwas_data), dataset2 = as.list(eqtl_data))

    # shorten the sublist named id to just 1 element
    out$dataset1$id <- out$dataset1$id[1]
    out$dataset2$id <- out$dataset2$id[1]
    # str(out)

    # plot the data
    temp <- coloc_to_gassocplot(out)
    # str(temp)
    gassocplot::stack_assoc_plot(temp$markers, temp$z, temp$corr, traits = temp$traits)
}
